In [1]:
from lumibot.brokers import Alpaca
from lumibot.backtesting import YahooDataBacktesting
from lumibot.strategies.strategy import Strategy
from lumibot.traders import Trader
from alpaca_trade_api import REST
from timedelta import Timedelta

from datetime import datetime
from pathlib import Path

2024-09-22 00:47:12 | lumibot.credentials | WARNING | No .env file found. This is ok if you are using environment variables or secrets (like on Replit, AWS, etc), but if you are not, please create a .env file in the root directory of the project.


In [2]:
notebook_dir = Path('.')
file_path = notebook_dir.absolute().parent / 'data' / 'secrets.txt'

secret_dict = {}
with open(file_path, 'r') as file:
    for line in file:
        line = line.strip()
        if '=' in line:
            key, value = line.split('=', 1)
            key = key.strip()
            value = value.strip().strip('"')
            secret_dict[key] = value

# print(secret_dict)

In [3]:
API_KEY = secret_dict['API_KEY']
API_SECRET = secret_dict['API_SECRET']
BASE_URL = secret_dict['BASE_URL']

In [4]:
ALPACA_CREDS = {
    "API_KEY":API_KEY,
    "API_SECRET":API_SECRET,
    "PAPER" : True
}

In [5]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
from typing import Tuple 
device = "cuda:0" if torch.cuda.is_available() else "cpu"

tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert").to(device)
labels = ["positive", "negative", "neutral"]

def estimate_sentiment(news):
    if news:
        tokens = tokenizer(news, return_tensors="pt", padding=True).to(device)

        result = model(tokens["input_ids"], attention_mask=tokens["attention_mask"])[
            "logits"
        ]
        result = torch.nn.functional.softmax(torch.sum(result, 0), dim=-1)
        probability = result[torch.argmax(result)]
        sentiment = labels[torch.argmax(result)]
        return probability, sentiment
    else:
        return 0, labels[-1]


if __name__ == "__main__":
    tensor, sentiment = estimate_sentiment(['markets responded negatively to the news!','traders were displeased!'])
    print(tensor, sentiment)
    print(torch.cuda.is_available())

C:\Users\akhan147\Anaconda3\envs\trader\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\akhan147\Anaconda3\envs\trader\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


tensor(0.9979, grad_fn=<SelectBackward0>) negative
False


In [6]:
class MLTrader(Strategy):
    # We start the bot. Initialize method will run once.
    def initialize(self, symbol:str='SPY', 
                   cash_at_risk:float=0.5
                  ):
        self.symbol=symbol
        self.sleeptime = '24H' # how frequently we want to trade
        self.last_trade = None # capture last trade. in case undo some sell/buy
        self.cash_at_risk = cash_at_risk # how much of our cash balance we want to risk at every trade
        self.api = REST(base_url= BASE_URL, 
                        key_id=API_KEY,
                        secret_key=API_SECRET
                       )
        
    def position_sizing(self):
        cash = self.get_cash()
        last_price = self.get_last_price(self.symbol)
        quantity = round(cash * self.cash_at_risk / last_price, 0)

        return cash, last_price, quantity

    def get_dates(self):
        today = self.get_datetime()
        three_days_prior = today - Timedelta(days=3)
        return today.strftime('%Y-%m-%d'), three_days_prior.strftime('%Y-%m-%d')
        
    
    def get_sentiment(self):
        today, three_days_prior = self.get_dates()
        news = self.api.get_news(symbol=self.symbol,
                                 start = three_days_prior,
                                 end = today
                                )
        news = [ev.__dict__["_raw"]["headline"] for ev in news]
        probability, sentiment = estimate_sentiment(news)
        return probability, sentiment
        
    
    # every time we get a tick
    def on_trading_iteration(self):
        cash, last_price, quantity = self.position_sizing()
        probability, sentiment = self.get_sentiment()

        if cash > last_price:
            if sentiment == 'positive' and probability > 0.999:
                if self.last_trade == "sell": 
                    self.sell_all() 
                order = self.create_order(
                    self.symbol,
                    quantity, # how many we want to buy
                    'buy',
                    type = 'bracket', # options: limit, bracket
                    take_profit_price = last_price*1.20,
                    stop_loss_price = last_price*0.95
                
                )
                self.submit_order(order)
                self.last_trade = 'buy'


        elif sentiment == 'negative' and probability > 0.999:
            if self.last_trade == "buy": 
                self.sell_all() 
            order = self.create_order(
                self.symbol,
                quantity, # how many we want to buy
                'sell',
                type = 'bracket', # options: limit, bracket
                take_profit_price = last_price*0.8,
                stop_loss_price = last_price*1.05
            
            )
            self.submit_order(order)
            self.last_trade = 'sell'

In [7]:
# add image for bracket loss

In [8]:
start_date = datetime(2020, 1, 1)
end_date = datetime(2023, 12, 31)

In [9]:
broker = Alpaca(ALPACA_CREDS)

2024-09-22 00:47:17 | asyncio | INFO | [unknown] Waiting for the socket stream connection to be established, 
                method _stream_established must be called
2024-09-22 00:47:18 | alpaca.trading.stream | INFO | started trading stream
2024-09-22 00:47:18 | alpaca.trading.stream | INFO | starting trading websocket connection


2024-09-22 00:49:37,145: alpaca.trading.stream: INFO: connected to: wss://paper-api.alpaca.markets/stream


In [10]:
strategy = MLTrader(name='mlstrat',
                    broker=broker,
                    parameters = {'symbol' : 'SPY',
                                  'cash_at_risk':0.5 # higher number means more cash per trade --> more risk taken
                                  
                                 
                                 }
                   )

In [12]:
strategy.backtest(YahooDataBacktesting,
                 start_date,
                 end_date,
                 parameters={})

Starting backtest for MLTrader...
Progress |██████████████████████████████████████████████████████████████████| 100.00%  [Elapsed: 0:06:46 ETA: 0:00:00] 
Creating trades plot...

Creating indicators plot...

Creating tearsheet...


{'cagr': 0.037902653648064666,
 'volatility': 0.06188076707275991,
 'sharpe': -0.22991545244258188,
 'max_drawdown': {'drawdown': 0.10640172510503991,
  'date': Timestamp('2023-10-26 09:30:00-0400', tz='America/New_York')},
 'romad': 0.3562221722499999,
 'total_return': 0.160095617239991}